In [1]:
import numpy as np
import pandas as pd
import pybithumb
import os
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from Funcs_CNN4 import obv, macd, cmo, rsi
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)

Scaler = MinMaxScaler()

dir = 'ohlcv/'
ohlcv_list = os.listdir(dir)


def min_max_scaler(price):
    Scaler = MinMaxScaler()
    Scaler.fit(price)

    return Scaler.transform(price)


def low_high(Coin, input_data_length, ip_limit=None, trade_limit=None, crop_size=300, lowhigh_point=None):

    #   거래 제한은 고점과 저점을 분리한다.

    #   User-Agent Configuration
    #   IP - Change
    if ip_limit is None:
        ohlcv_excel = pybithumb.get_ohlcv(Coin, 'KRW', 'minute1')
    else:
        ohlcv_excel = pybithumb.get_ohlcv(Coin, 'KRW', 'minute1', 'proxyon')

    price_gap = ohlcv_excel.close.max() / ohlcv_excel.close.min()
    if (price_gap < 1.07) and (trade_limit is not None):
        return None, None, None, None

    ohlcv_excel['OBV'] = obv(ohlcv_excel)

    if lowhigh_point is not None:
        check_span = 20
        trade_state = [np.NaN] * len(ohlcv_excel)
        for i in range(check_span, len(ohlcv_excel) - check_span):
            #   저점
            if ohlcv_excel['close'][i - check_span:i].min() >= ohlcv_excel['close'][i]:
                if ohlcv_excel['close'][i + 1:i + 1 + check_span].min() >= ohlcv_excel['close'][i]:
                    # if ohlcv_excel['close'][i - check_span:i + 1 + check_span].value_counts().sort_index().iloc[0] <= 3:
                    if 1 not in trade_state[i - check_span:i]:
                        trade_state[i] = 1
                    else:
                        trade_state[i] = 0
                    # else:
                    #     trade_state[i] = 0
                else:
                    trade_state[i] = 0
            #   고점
            elif ohlcv_excel['close'][i - check_span:i].max() <= ohlcv_excel['close'][i]:
                if ohlcv_excel['close'][i + 1:i + 1 + check_span].max() <= ohlcv_excel['close'][i]:
                    # if ohlcv_excel['close'][i - check_span:i + 1 + check_span].value_counts().sort_index().iloc[-1] <= 3:
                    if 2 not in trade_state[i - check_span:i]:
                        trade_state[i] = 2
                    else:
                        trade_state[i] = 0
                # else:
                #     trade_state[i] = 0
                else:
                    trade_state[i] = 0

            #   거래 안함
            else:
                trade_state[i] = 0

        ohlcv_excel['trade_state'] = trade_state

        #   현재가가 이전 저점보다 0.1 이상 크지 않으면 거래하지 않는다.
        low_index = None
        for i in range(len(trade_state)):
            if trade_state[-i] == 1:
                low_index = i

    closeprice = ohlcv_excel['close'].iloc[-1]
    datetime_list = ohlcv_excel.index.values

    # ----------- dataX, dataY 추출하기 -----------#
    #   OBV :
    ohlcv_data = ohlcv_excel.values[1:].astype(np.float)

    scaled_close = min_max_scaler(ohlcv_data[:, [1]])
    prev_low = scaled_close[-low_index]
    pres_low = scaled_close[-1]
    if pres_low - prev_low < 0.1:
        return None, None, None, None

    # 결측 데이터 제외
    if len(ohlcv_data) != 0:

        #          데이터 전처리         #
        #   Fixed X_data    #
        # price = ohlcv_data[:, :4]
        # volume = ohlcv_data[:, [4]]
        # OBV = ohlcv_data[:, [-1]]
        #
        # scaled_price = min_max_scaler(price)
        # scaled_volume = min_max_scaler(volume)
        # scaled_OBV = min_max_scaler(OBV)
        # print(scaled_MA60.shape)

        # x = np.concatenate((scaled_price, scaled_volume, scaled_OBV), axis=1)  # axis=1, 세로로 합친다
        # print(x.shape)
        # print(ohlcv_data.shape)
        # quit()

        # if (x[-1][1] > 0.3) and (trade_limit is not None):
        #     return None, None

        # print(x.shape)  # (258, 6)
        # quit()

        dataX = []  # input_data length 만큼 담을 dataX 그릇
        for i in range(crop_size, len(ohlcv_data) + 1):  # 마지막 데이터까지 다 긇어모은다.
            group_x = ohlcv_data[i - crop_size: i]
            scaled_price = min_max_scaler(group_x[:, :4])
            # scaled_volume = min_max_scaler(group_x[:, [4]])
            # scaled_OBV = min_max_scaler(group_x[:, [-1]])
            # x = np.concatenate((scaled_price, scaled_volume, scaled_OBV), axis=1) + sudden_death  # axis=1, 세로로 합친다
            x = scaled_price  # axis=1, 세로로 합친다
            group_x = x[-input_data_length:]
            # print(group_x[-1:, 1:2])
            # print(group_x.shape)
            # print(group_x.max())
            dataX.append(group_x)  # dataX 리스트에 추가

        if len(dataX) < 100:
            return None, None, None, None

        # quit()
        X_test = np.array(dataX)
        row = X_test.shape[1]
        col = X_test.shape[2]

        X_test = X_test.astype('float32').reshape(-1, row, col, 1)
        # closeprice = np.roll(np.array(list(map(lambda x: x[-1][[1]][0], X_test))), -1)
        # plt.plot(closeprice)
        # plt.show()

        return X_test, closeprice, min_max_scaler(ohlcv_data[crop_size:, [1]]), datetime_list


def made_x(file, input_data_length, model_num, check_span, get_fig, crop_size=None, sudden_death=0):

    ohlcv_excel = pd.read_excel(dir + file, index_col=0)

    ohlcv_excel['MA60'] = ohlcv_excel['close'].rolling(60).mean()
    ohlcv_excel['CMO'] = cmo(ohlcv_excel)
    ohlcv_excel['OBV'] = obv(ohlcv_excel)
    ohlcv_excel['RSI'] = rsi(ohlcv_excel)
    macd(ohlcv_excel)

    # print(ohlcv_excel.info)
    # quit()

    #   이후 check_span 데이터와 현재 포인트를 비교해서 현재 포인트가 저가인지 고가인지 예측한다.
    #   진입, 저점, 고점, 거래 안함의 y_label 인 trade_state  >> [1, 2, 0]
    #   저점과 고점은 최대 3개의 중복 값을 허용한다.
    trade_state = [np.NaN] * len(ohlcv_excel)
    max_index_list = list()
    for i in range(1, len(ohlcv_excel) - check_span):

        if (ohlcv_excel['close'][i - 2] <= ohlcv_excel['MA60'][i - 2]) and (ohlcv_excel['close'][i - 1] > ohlcv_excel['MA60'][i - 1]):
            # trade_state[i] = 1

            #   진입
            for i2 in range(i + 1, i + check_span):
                if ohlcv_excel['close'][i2] <= ohlcv_excel['MA60'][i2]:
                    break
                if i2 == i + check_span - 1:

                    #   이탈
                    #   진입부터 MA선 밑으로 내려가는 SPAN POINT 찾기
                    for i3 in range(i, len(ohlcv_excel) - check_span):
                        # print(ohlcv_excel['close'][i3], ohlcv_excel['MA60'][i3])
                        if ohlcv_excel['close'][i3] <= ohlcv_excel['MA60'][i3]:
                            break

                    #   진입부터 SPAN_POINT 까지의 MAX값 INDEX 찾기
                    for i4 in range(i, i3):
                        if ohlcv_excel['close'][i4] == ohlcv_excel['close'][i:i3].max():
                            break
                    # print(i, i3, i4)
                    trade_state[i] = 1
                    max_index_list.append(i4)

        #   거래 안함
        else:
            trade_state[i] = 0

    for index in max_index_list:
        trade_state[index] = 2
    ohlcv_excel['trade_state'] = trade_state
    # print(trade_state)
    # quit()

    # ----------- dataX, dataY 추출하기 -----------#
    # print(ohlcv_excel.info())
    # quit()
    # ohlcv_excel.to_excel('test.xlsx')

    # NaN 제외하고 데이터 자르기 (데이터가 PIXEL 로 들어간다고 생각하면 된다)
    #   OBV : -CHECK_SPAN
    ohlcv_data = ohlcv_excel.values[sum(ohlcv_excel.MA60.isna()): -check_span].astype(np.float)
    # ohlcv_data = ohlcv_excel.values[sum(ohlcv_excel.MACD_Signal.isna()): -check_span].astype(np.float)
    # ohlcv_data = ohlcv_excel.values[1: -check_span].astype(np.float)
    # ohlcv_data = ohlcv_excel.values[sum(ohlcv_excel.CMO.isna()): -check_span].astype(np.float)
    # ohlcv_data = ohlcv_excel.values[sum(ohlcv_excel.RSI.isna()): -check_span].astype(np.float)

    # print(pd.DataFrame(ohlcv_data).info())
    # print(pd.DataFrame(ohlcv_data).to_excel('test.xlsx'))
    # print(list(map(float, ohlcv_data[0])))
    # print(ohlcv_data.shape)
    # quit()

    #       출력되는 Y 데이터 길이를 동일화 시킨다.     #
    # ohlcv_data = ohlcv_data[-300:]

    # 결측 데이터 제외
    if len(ohlcv_data) != 0:

        #          데이터 전처리         #
        #   Fixed X_data    #
#         price = ohlcv_data[:, :4]
#         volume = ohlcv_data[:, [4]]
#         MA60 = ohlcv_data[:, [-8]]
#         CMO = ohlcv_data[:, [-7]]
#         OBV = ohlcv_data[:, [-6]]
#         RSI = ohlcv_data[:, [-5]]
#         MACD = ohlcv_data[:, [-4]]
#         MACD_Signal = ohlcv_data[:, [-3]]
#         MACD_OSC = ohlcv_data[:, [-2]]

        #   Flexible Y_data    #
        trade_state = ohlcv_data[:, [-1]]

        # scaled_price = min_max_scaler(price)
        # scaled_volume = min_max_scaler(volume)
        # scaled_CMO = min_max_scaler(CMO)
        # scaled_OBV = min_max_scaler(OBV)
        # scaled_RSI = min_max_scaler(RSI)

        # x = np.concatenate((scaled_price, scaled_volume, scaled_OBV), axis=1)  # axis=1, 세로로 합친다
        # x = scaled_price
        y = trade_state
        # print(x.shape, y_low.shape)  # (258, 6) (258, 1)
        # quit()

        dataX = []  # input_data length 만큼 담을 dataX 그릇
        dataY = []  # Target 을 담을 그릇
        for i in range(crop_size, len(ohlcv_data)):  # 마지막 데이터까지 다 긇어모은다.
            group_x = ohlcv_data[i - crop_size: i]
            group_y = y[i]
#             price = group_x[:, :4]
            pricema = group_x[:, [0, 1, 2, 3, 5]]
            volume = group_x[:, [4]]
#             MA60 = group_x[:, [-8]]
            CMO = group_x[:, [-7]]
            OBV = group_x[:, [-6]]
            RSI = group_x[:, [-5]]
            MACD = group_x[:, [-4]]
            MACD_Signal = group_x[:, [-3]]
            MACD_OSC = group_x[:, [-2]]
#             scaled_price = min_max_scaler(price)
            scaled_pricema = min_max_scaler(pricema)
            scaled_volume = min_max_scaler(volume)
#             scaled_MA60 = min_max_scaler(MA60)
            scaled_CMO = min_max_scaler(CMO)
            scaled_OBV = min_max_scaler(OBV)
            scaled_RSI = min_max_scaler(RSI)
            scaled_MACD = min_max_scaler(MACD)
            scaled_MACD_Signal = min_max_scaler(MACD_Signal)
            scaled_MACD_OSC = min_max_scaler(MACD_OSC)
            x = np.concatenate((scaled_pricema[:, :-1], scaled_volume, 
                                scaled_pricema[:, [-1]], scaled_CMO, scaled_OBV,
                                scaled_RSI,
                                scaled_MACD, scaled_MACD_Signal,
                                scaled_MACD_OSC), axis=1)
            # x = scaled_x + sudden_death  # axis=1, 세로로 합친다
            group_x = x[-input_data_length:]
            # print(group_x[0])
            # quit()

            #   데이터 값에 결측치가 존재하는 경우 #
            if (sum(sum(np.isnan(group_x))) > 0) or (sum(np.isnan(group_y)) > 0):
                continue

            dataX.append(group_x)  # dataX 리스트에 추가
            dataY.append(group_y)

        if len(dataX) < 100:
            print('len(dataX) < 100')
            return None, None, None

        # X_test = np.array(dataX)
        # print(X_test.shape)
        # quit()
        # row = X_test.shape[1]
        # col = X_test.shape[2]
        #
        # X_test = X_test.astype('float32').reshape(-1, row, col, 1)
        # print(X_test.shape)

        #       Exstracting fiexd X_data       #
        sliced_ohlcv = min_max_scaler(ohlcv_data[crop_size:, :x.shape[1]])

        #                      Get Figure                     #
        if get_fig == 1:
            spanlist_low = []
            spanlist_high = []

            for m in range(len(trade_state)):
                if (trade_state[m] > 0.5) and (trade_state[m] < 1.5):
                    if m + 1 < len(trade_state):
                        spanlist_low.append((m, m + 1))
                    else:
                        spanlist_low.append((m - 1, m))

            for m in range(len(trade_state)):
                if (trade_state[m] > 1.5) and (trade_state[m] < 2.5):
                    if m + 1 < len(trade_state):
                        spanlist_high.append((m, m + 1))
                    else:
                        spanlist_high.append((m - 1, m))

            # ----------- 인덱스 초기화 됨 -----------#

            # ----------- 공통된 Chart 그리기 -----------#

            plt.subplot(211)
            plt.plot((ohlcv_data[:, [1]]), 'gold', label='close')
            plt.plot((ohlcv_data[:, [5]]), 'b', label='ma')
            plt.legend(loc='upper right')
            for i in range(len(spanlist_low)):
                plt.axvspan(spanlist_low[i][0], spanlist_low[i][1], facecolor='c', alpha=0.7)

            plt.subplot(212)
            plt.plot((ohlcv_data[:, [1]]), 'gold', label='close')
            plt.plot((ohlcv_data[:, [5]]), 'b', label='ma')
            plt.legend(loc='upper right')
            for i in range(len(spanlist_high)):
                plt.axvspan(spanlist_high[i][0], spanlist_high[i][1], facecolor='m', alpha=0.7)

            Date = file.split()[0]
            Coin = file.split()[1].split('.')[0]
            plt.savefig('./Figure_data/%s_%s/%s %s.png' % (input_data_length, model_num, Date, Coin), dpi=500)
            plt.close()
            # plt.show()
            # ----------- Chart 그리기 -----------#

        return dataX, dataY, sliced_ohlcv


if __name__ == '__main__':

    # ----------- Params -----------#
    input_data_length = 30
    model_num = 63

    #       Make folder      #
    try:
        os.mkdir('./Figure_data/%s_%s/' % (input_data_length, model_num))

    except Exception as e:
        pass
    check_span = 40
    get_fig = 0

    Made_X = []
    Made_Y = []

#     ohlcv_list = ['2020-01-10 ETH ohlcv.xlsx']

    for file in ohlcv_list:
        try:
            if int(file.split()[0].split('-')[1]) != 1:
                continue


            result = made_x(file, input_data_length, model_num, check_span, get_fig, crop_size=input_data_length)
            # result = low_high('dvp'.upper(), input_data_length)
            # print(result)
            # quit()

            # ------------ 데이터가 있으면 dataX, dataY 병합하기 ------------#
            if result is not None:
                if result[0] is not None:

                    Made_X += result[0]
                    Made_Y += result[1]

                # 누적 데이터량 표시
                print(file, len(Made_X))
                
        except Exception as e:
            pass

    # SAVING X, Y
    X = np.array(Made_X)
    Y = np.array(Made_Y)
    np.save('./Made_X/Made_X %s_%s' % (input_data_length, model_num), X)
    np.save('./Made_X/Made_Y %s_%s' % (input_data_length, model_num), Y)
    # np.save('./Made_X_high/Made_Y %s_%s' % (input_data_length, model_num), Y_high)



2020-01-16 XLM ohlcv.xlsx 938
2020-01-23 TRX ohlcv.xlsx 1531
2020-01-31 AMO ohlcv.xlsx 2000
len(dataX) < 100
2020-01-19 LBA ohlcv.xlsx 2000
2020-01-11 FX ohlcv.xlsx 2250
2020-01-22 LUNA ohlcv.xlsx 3268
2020-01-16 LINK ohlcv.xlsx 4186
len(dataX) < 100
2020-01-28 PLY ohlcv.xlsx 4186
2020-01-13 FCT ohlcv.xlsx 4337
2020-01-20 IPX ohlcv.xlsx 5562
len(dataX) < 100
2020-01-27 ITC ohlcv.xlsx 5562
2020-01-20 SNT ohlcv.xlsx 5759
2020-01-29 TMTG ohlcv.xlsx 6601
len(dataX) < 100
2020-01-25 VET ohlcv.xlsx 6601
2020-01-21 EOS ohlcv.xlsx 7662
2020-01-24 FAB ohlcv.xlsx 7775
2020-01-22 TRV ohlcv.xlsx 7904
2020-01-28 TRX ohlcv.xlsx 8465
len(dataX) < 100
2020-01-20 CMT ohlcv.xlsx 8465
2020-01-22 BTG ohlcv.xlsx 9388
2020-01-13 DAC ohlcv.xlsx 10429
2020-01-25 REP ohlcv.xlsx 10694
2020-01-11 APIS ohlcv.xlsx 10855
2020-01-26 XRP ohlcv.xlsx 11812
2020-01-13 APIS ohlcv.xlsx 11916
len(dataX) < 100
2020-01-31 CMT ohlcv.xlsx 11916
2020-01-16 ETH ohlcv.xlsx 13163
len(dataX) < 100
2020-01-22 STRAT ohlcv.xlsx 13163


len(dataX) < 100
2020-01-30 BAT ohlcv.xlsx 79505
2020-01-25 ENJ ohlcv.xlsx 80325
2020-01-30 BCH ohlcv.xlsx 81401
2020-01-30 DAC ohlcv.xlsx 82408
len(dataX) < 100
2020-01-20 POWR ohlcv.xlsx 82408
2020-01-12 LINK ohlcv.xlsx 82840
len(dataX) < 100
2020-01-30 THETA ohlcv.xlsx 82840
2020-01-28 AMO ohlcv.xlsx 83319
len(dataX) < 100
2020-01-15 MXC ohlcv.xlsx 83319
2020-01-26 EOS ohlcv.xlsx 84192
len(dataX) < 100
2020-01-29 PIVX ohlcv.xlsx 84192
2020-01-13 OCN ohlcv.xlsx 84456
2020-01-31 FX ohlcv.xlsx 84842
len(dataX) < 100
2020-01-21 HC ohlcv.xlsx 84842
len(dataX) < 100
2020-01-13 PIVX ohlcv.xlsx 84842
2020-01-22 XLM ohlcv.xlsx 85435
2020-01-24 BTT ohlcv.xlsx 85568
len(dataX) < 100
2020-01-23 HDAC ohlcv.xlsx 85568
2020-01-24 BTC ohlcv.xlsx 86774
len(dataX) < 100
2020-01-12 PPT ohlcv.xlsx 86774
2020-01-29 LTC ohlcv.xlsx 87430
2020-01-23 ADA ohlcv.xlsx 87851
2020-01-23 STEEM ohlcv.xlsx 88181
2020-01-11 BCH ohlcv.xlsx 89039
2020-01-27 LUNA ohlcv.xlsx 89178
len(dataX) < 100
2020-01-28 WAVES ohlcv

2020-01-25 FCT ohlcv.xlsx 148957
2020-01-15 ADA ohlcv.xlsx 149324
2020-01-15 XMR ohlcv.xlsx 149426
len(dataX) < 100
2020-01-31 VET ohlcv.xlsx 149426
len(dataX) < 100
2020-01-23 ZRX ohlcv.xlsx 149426
len(dataX) < 100
2020-01-24 WAXP ohlcv.xlsx 149426
2020-01-27 DAD ohlcv.xlsx 149527
2020-01-11 SOC ohlcv.xlsx 150465
len(dataX) < 100
2020-01-30 HDAC ohlcv.xlsx 150465
len(dataX) < 100
2020-01-30 XMR ohlcv.xlsx 150465
len(dataX) < 100
2020-01-19 XVG ohlcv.xlsx 150465
2020-01-12 STEEM ohlcv.xlsx 150614
2020-01-31 RDN ohlcv.xlsx 150735
2020-01-22 FAB ohlcv.xlsx 151127
len(dataX) < 100
2020-01-15 RDN ohlcv.xlsx 151127
len(dataX) < 100
2020-01-30 VET ohlcv.xlsx 151127
len(dataX) < 100
2020-01-24 RNT ohlcv.xlsx 151127
2020-01-30 XLM ohlcv.xlsx 151841
len(dataX) < 100
2020-01-29 CHR ohlcv.xlsx 151841
2020-01-12 REP ohlcv.xlsx 152100
2020-01-15 OGO ohlcv.xlsx 152364
len(dataX) < 100
2020-01-14 LAMB ohlcv.xlsx 152364
len(dataX) < 100
2020-01-13 MIX ohlcv.xlsx 152364
2020-01-28 BSV ohlcv.xlsx 153516

len(dataX) < 100
2020-01-23 XVG ohlcv.xlsx 208384
len(dataX) < 100
2020-01-31 ETZ ohlcv.xlsx 208384
2020-01-30 TMTG ohlcv.xlsx 209395
len(dataX) < 100
2020-01-24 POWR ohlcv.xlsx 209395
len(dataX) < 100
2020-01-28 WOM ohlcv.xlsx 209395
len(dataX) < 100
2020-01-26 DAD ohlcv.xlsx 209395
len(dataX) < 100
2020-01-27 HDAC ohlcv.xlsx 209395
2020-01-31 CON ohlcv.xlsx 209826
2020-01-25 TRX ohlcv.xlsx 210360
2020-01-21 AMO ohlcv.xlsx 210855
2020-01-27 BSV ohlcv.xlsx 211905
2020-01-27 TRX ohlcv.xlsx 212251
2020-01-21 PLY ohlcv.xlsx 212433
len(dataX) < 100
2020-01-24 DVP ohlcv.xlsx 212433
len(dataX) < 100
2020-01-24 VET ohlcv.xlsx 212433
2020-01-16 ELF ohlcv.xlsx 212749
2020-01-14 RNT ohlcv.xlsx 212898
2020-01-28 ETH ohlcv.xlsx 213884
len(dataX) < 100
2020-01-24 SNT ohlcv.xlsx 213884
len(dataX) < 100
2020-01-29 GXC ohlcv.xlsx 213884
len(dataX) < 100
2020-01-24 IOST ohlcv.xlsx 213884
len(dataX) < 100
2020-01-22 ETZ ohlcv.xlsx 213884
2020-01-19 ENJ ohlcv.xlsx 214043
2020-01-25 TMTG ohlcv.xlsx 214728

2020-01-27 BTC ohlcv.xlsx 274405
len(dataX) < 100
2020-01-30 ORBS ohlcv.xlsx 274405
len(dataX) < 100
2020-01-22 THETA ohlcv.xlsx 274405
len(dataX) < 100
2020-01-14 TRV ohlcv.xlsx 274405
2020-01-27 LTC ohlcv.xlsx 274676
2020-01-30 ZEC ohlcv.xlsx 274993
2020-01-12 EOS ohlcv.xlsx 276111
2020-01-12 BSV ohlcv.xlsx 277250
len(dataX) < 100
2020-01-15 ITC ohlcv.xlsx 277250
len(dataX) < 100
2020-01-29 ITC ohlcv.xlsx 277250
2020-01-11 LINK ohlcv.xlsx 277684
2020-01-25 QTUM ohlcv.xlsx 278141
len(dataX) < 100
2020-01-22 HDAC ohlcv.xlsx 278141
2020-01-24 WPX ohlcv.xlsx 279028
2020-01-11 BHP ohlcv.xlsx 279463
len(dataX) < 100
2020-01-22 DAD ohlcv.xlsx 279463
2020-01-19 BTC ohlcv.xlsx 280717
2020-01-24 ELF ohlcv.xlsx 280983
2020-01-15 AOA ohlcv.xlsx 281180
2020-01-14 BSV ohlcv.xlsx 282057
len(dataX) < 100
2020-01-13 WET ohlcv.xlsx 282057
len(dataX) < 100
2020-01-25 STEEM ohlcv.xlsx 282057
2020-01-31 WPX ohlcv.xlsx 282648
2020-01-15 HYC ohlcv.xlsx 282750
len(dataX) < 100
2020-01-23 FZZ ohlcv.xlsx 2827

2020-01-25 AOA ohlcv.xlsx 340621
2020-01-28 QTUM ohlcv.xlsx 341109
2020-01-25 EOS ohlcv.xlsx 342075
len(dataX) < 100
2020-01-11 LOOM ohlcv.xlsx 342075
len(dataX) < 100
2020-01-21 ETZ ohlcv.xlsx 342075
len(dataX) < 100
2020-01-15 RNT ohlcv.xlsx 342075
2020-01-14 TRX ohlcv.xlsx 342635
2020-01-20 ETH ohlcv.xlsx 343785
len(dataX) < 100
2020-01-25 BZNT ohlcv.xlsx 343785
len(dataX) < 100
2020-01-29 DAD ohlcv.xlsx 343785
len(dataX) < 100
2020-01-20 MCO ohlcv.xlsx 343785
2020-01-23 FAB ohlcv.xlsx 344025
len(dataX) < 100
2020-01-31 WAXP ohlcv.xlsx 344025
2020-01-11 ETZ ohlcv.xlsx 344558
2020-01-29 CON ohlcv.xlsx 345291
2020-01-30 EOS ohlcv.xlsx 346401
len(dataX) < 100
2020-01-27 INS ohlcv.xlsx 346401
len(dataX) < 100
2020-01-30 LAMB ohlcv.xlsx 346401
2020-01-23 ABT ohlcv.xlsx 346583
len(dataX) < 100
2020-01-22 BHP ohlcv.xlsx 346583
len(dataX) < 100
2020-01-19 RNT ohlcv.xlsx 346583
2020-01-24 EOS ohlcv.xlsx 347438
2020-01-21 ETC ohlcv.xlsx 348430
len(dataX) < 100
2020-01-29 CTXC ohlcv.xlsx 34843

len(dataX) < 100
2020-01-12 XEM ohlcv.xlsx 402266
2020-01-16 HC ohlcv.xlsx 402780
2020-01-14 SOC ohlcv.xlsx 403358
len(dataX) < 100
2020-01-21 PCM ohlcv.xlsx 403358
2020-01-16 NPXS ohlcv.xlsx 403551
2020-01-25 XSR ohlcv.xlsx 404239
2020-01-16 OMG ohlcv.xlsx 404577
len(dataX) < 100
2020-01-19 WICC ohlcv.xlsx 404577
len(dataX) < 100
2020-01-19 PLY ohlcv.xlsx 404577
len(dataX) < 100
2020-01-11 LRC ohlcv.xlsx 404577
2020-01-22 LTC ohlcv.xlsx 405062
len(dataX) < 100
2020-01-22 XEM ohlcv.xlsx 405062
len(dataX) < 100
2020-01-29 WAVES ohlcv.xlsx 405062
2020-01-31 BCH ohlcv.xlsx 406075
len(dataX) < 100
2020-01-14 XMR ohlcv.xlsx 406075
2020-01-16 TRV ohlcv.xlsx 406240
len(dataX) < 100
2020-01-13 POWR ohlcv.xlsx 406240
2020-01-16 XSR ohlcv.xlsx 407049
2020-01-11 TMTG ohlcv.xlsx 408205
2020-01-22 LINK ohlcv.xlsx 408634
len(dataX) < 100
2020-01-16 QKC ohlcv.xlsx 408634
2020-01-30 BCD ohlcv.xlsx 409175
len(dataX) < 100
2020-01-13 WOM ohlcv.xlsx 409175
2020-01-15 BCD ohlcv.xlsx 409973
len(dataX) < 10